In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from math import sqrt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten


from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import yfinance as yf

In [25]:
# raccolata dati

t = yf.Ticker("AAPL")
dati = t.history(period = "10Y")
dati_storici = dati["Close"].values

In [26]:
dati_storici

array([ 10.6821413 ,  10.61089516,  10.62901211, ..., 126.84999847,
       125.27999878, 124.61000061])

In [27]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
         # find the end of this pattern
         end_ix = i + n_steps_in
         out_end_ix = end_ix + n_steps_out
         # check if we are beyond the sequence
         if out_end_ix > len(sequence):
             break
         # gather input and output parts of the pattern
         seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
         X.append(seq_x)
         y.append(seq_y)
    return np.array(X), np.array(y)

# choose a number of time steps
n_steps_in, n_steps_out = 7, 7

# split into samples
X, y = split_sequence(dati_storici, n_steps_in, n_steps_out)
count = 0
for i in range(len(X)):
    print(X[i], y[i])
    count +=1
    if count>5: break

[10.6821413  10.61089516 10.62901211 10.54732132 10.38148594 10.19721603
 10.20336151] [10.18032742 10.0086565  10.03015518 10.20950508 10.03475952  9.98592949
  9.8354454 ]
[10.61089516 10.62901211 10.54732132 10.38148594 10.19721603 10.20336151
 10.18032742] [10.0086565  10.03015518 10.20950508 10.03475952  9.98592949  9.8354454
  9.68373299]
[10.62901211 10.54732132 10.38148594 10.19721603 10.20336151 10.18032742
 10.0086565 ] [10.03015518 10.20950508 10.03475952  9.98592949  9.8354454   9.68373299
  9.99022865]
[10.54732132 10.38148594 10.19721603 10.20336151 10.18032742 10.0086565
 10.03015518] [10.20950508 10.03475952  9.98592949  9.8354454   9.68373299  9.99022865
  9.90761566]
[10.38148594 10.19721603 10.20336151 10.18032742 10.0086565  10.03015518
 10.20950508] [10.03475952  9.98592949  9.8354454   9.68373299  9.99022865  9.90761566
 10.17234421]
[10.19721603 10.20336151 10.18032742 10.0086565  10.03015518 10.20950508
 10.03475952] [ 9.98592949  9.8354454   9.68373299  9.99022

In [28]:
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
y = y.reshape((y.shape[0], y.shape[1], n_features))

In [29]:
train_size = 1800
train_X, test_X = X[:train_size], X[train_size:]
train_y, test_y = y[:train_size], y[train_size:]
print(len(train_X), len(train_y), len(test_X), len(test_y))

1800 1800 704 704


In [30]:
# define model
model = Sequential()
model.add(LSTM(20, activation='relu', input_shape=(n_steps_in, n_features))) 
model.add(RepeatVector(n_steps_out))
model.add(LSTM(20, activation='relu', return_sequences=True)) 
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse' )
# fit model
model.fit(train_X, train_y, epochs=200, verbose=2)

Epoch 1/200
57/57 - 2s - loss: 289.4960
Epoch 2/200
57/57 - 0s - loss: 20.2613
Epoch 3/200
57/57 - 0s - loss: 1.6573
Epoch 4/200
57/57 - 0s - loss: 1.0870
Epoch 5/200
57/57 - 0s - loss: 0.9449
Epoch 6/200
57/57 - 0s - loss: 0.9118
Epoch 7/200
57/57 - 0s - loss: 0.8420
Epoch 8/200
57/57 - 0s - loss: 0.8297
Epoch 9/200
57/57 - 0s - loss: 0.8223
Epoch 10/200
57/57 - 0s - loss: 0.7925
Epoch 11/200
57/57 - 0s - loss: 0.8133
Epoch 12/200
57/57 - 0s - loss: 0.7742
Epoch 13/200
57/57 - 0s - loss: 0.8182
Epoch 14/200
57/57 - 0s - loss: 0.8642
Epoch 15/200
57/57 - 0s - loss: 0.7841
Epoch 16/200
57/57 - 0s - loss: 0.7649
Epoch 17/200
57/57 - 0s - loss: 0.7670
Epoch 18/200
57/57 - 0s - loss: 0.7528
Epoch 19/200
57/57 - 0s - loss: 0.7428
Epoch 20/200
57/57 - 0s - loss: 0.7551
Epoch 21/200
57/57 - 0s - loss: 0.7455
Epoch 22/200
57/57 - 0s - loss: 0.7418
Epoch 23/200
57/57 - 0s - loss: 0.7171
Epoch 24/200
57/57 - 0s - loss: 0.7197
Epoch 25/200
57/57 - 0s - loss: 0.7366
Epoch 26/200
57/57 - 0s - loss:

In [31]:
model.save("MSFT.h5")


from tensorflow import keras
model = keras.models.load_model('MSFT.h5')

In [32]:
input_x = np.array([20.04079437 ,19.57603645, 19.40775299, 19.15935516, 19.23948669, 19.27955055,19.18339348])
x_input = input_x.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)

In [24]:
yhat

array([[[19.278101],
        [19.485794],
        [19.4035  ],
        [19.503675],
        [19.54487 ],
        [19.554474],
        [19.576674]]], dtype=float32)